## Predicting Cell Health Phenotypes with Specific Feature Subsets

The contribution of specific feature groups and Cell Painting stains to model performance is a major question.

Here, we address this question by systematically removing stains and feature groups and observing impact on model performance.

In [1]:
import sys
import pathlib
import numpy as np
import pandas as pd

from sklearn.linear_model import SGDClassifier, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.exceptions import ConvergenceWarning

sys.path.insert(0, "../3.train/scripts/")
from ml_utils import CellHealthPredict, load_train_test

In [2]:
# Set constants
consensus = "modz"
data_dir = pathlib.Path("../3.train/data")
results_dir = pathlib.Path("../3.train/results")

coef_file = pathlib.Path(f"{results_dir}/all_model_coefficients_{consensus}.tsv")

output_dir = pathlib.Path("results")
output_dir.mkdir(exist_ok=True)
output_file = pathlib.Path(f"{output_dir}/feature_group_subset_removal_robustness_results_{consensus}.tsv.gz")

# Define which features to remove per group
feature_removal_ids = {
    "feature_group": [
        "Texture",
        "Intensity",
        "Correlation",
        "AreaShape",
        "Granularity",
        "RadialDistribution",
        "Neighbors"
    ],
    "channel": [
        "AGP",
        "DNA",
        "ER",
        "Mito",
        "RNA"
    ],
    "compartment": [
        "Cells",
        "Cytoplasm",
        "Nuclei"
    ]
}

In [3]:
# Get all features and define feature groups
feature_df = pd.read_csv(coef_file, sep="\t")

feature_info_split_df = (
    pd.concat(
        [pd.Series(x.split("_")) for x in feature_df.features],
        axis="columns"
    )
    .transpose()
)
feature_info_split_df.columns = [
    "compartment", "feature_group", "measurement", "channel", "parameter1", "parameter2"
]
feature_info_split_df = feature_info_split_df.assign(full_feature_name=feature_df.features)

print(feature_info_split_df.shape)
feature_info_split_df.head()

(949, 7)


,compartment,feature_group,measurement,channel,parameter1,parameter2,full_feature_name
0,Cells,AreaShape,Center,Y,NaN,NaN,Cells_AreaShape_Center_Y
1,Cells,AreaShape,Compactness,NaN,NaN,NaN,Cells_AreaShape_Compactness
2,Cells,AreaShape,Eccentricity,NaN,NaN,NaN,Cells_AreaShape_Eccentricity
3,Cells,AreaShape,Extent,NaN,NaN,NaN,Cells_AreaShape_Extent
4,Cells,AreaShape,Orientation,NaN,NaN,NaN,Cells_AreaShape_Orientation


In [4]:
# Set ML constants
# We will optimize each model independently
alphas = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
l1_ratios = [0.1, 0.12, 0.14, 0.16, 0.2, 0.3, 0.4, 0.5, 0.7, 0.8, 0.9]
n_folds = 5

parameters = {
    'regress__alpha': alphas,
    'regress__l1_ratio': l1_ratios
}

estimator = Pipeline(
    steps=[(
        "regress",
        ElasticNet(
            random_state=42,
            max_iter=50,
            tol=1e-3
        )
        
    )]
)

y_transform = "raw"
scoring = "r2"
decision_function = False
model_type = "Regression"

In [5]:
# Load training/test data
x_train_df, x_test_df, y_train_df, y_test_df = (
    load_train_test(data_dir=data_dir, drop_metadata=True, consensus=consensus)
)

# Setup target variable names and assert that there are 70 of them
cell_health_targets = y_train_df.columns.tolist()
assert len(cell_health_targets) == 70

print(x_train_df.shape)
x_train_df.head(3)

(303, 949)


,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_Extent,Cells_AreaShape_Orientation,Cells_AreaShape_Zernike_0_0,Cells_AreaShape_Zernike_1_1,Cells_AreaShape_Zernike_2_0,Cells_AreaShape_Zernike_2_2,Cells_AreaShape_Zernike_3_3,...,Nuclei_Texture_SumEntropy_RNA_5_0,Nuclei_Texture_SumVariance_AGP_20_0,Nuclei_Texture_SumVariance_AGP_5_0,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_DNA_20_0,Nuclei_Texture_SumVariance_DNA_5_0,Nuclei_Texture_Variance_AGP_5_0,Nuclei_Texture_Variance_DNA_10_0,Nuclei_Texture_Variance_DNA_20_0,Nuclei_Texture_Variance_DNA_5_0
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_340,0.029292,0.607142,-0.576461,-2.459725,0.587515,-1.183370,0.257162,0.741291,-3.732722,-0.012861,...,-2.964612,-2.131689,-2.073252,-2.623642,-2.597662,-2.869573,-2.367492,-3.110212,-3.150354,-2.901882
profile_6,-0.087900,0.186519,0.562200,-0.060381,-0.206748,-0.318174,0.682207,-0.121530,-0.266203,-0.494876,...,-0.030618,-0.285945,-0.297946,-0.254188,-0.020307,-0.190692,-0.358839,-0.388787,-0.318564,-0.311846
profile_79,0.233590,1.738854,1.829460,-1.535546,0.285378,-1.958437,-0.673058,-0.674018,0.430340,-0.227258,...,-1.216083,-1.014071,-1.006968,-0.763500,-0.436459,-0.727370,-0.968550,-0.793551,-0.722493,-0.738353


## Systematically drop features and save performance results

In [6]:
regression_results_list = []
for feature_category in feature_removal_ids:
    feature_removal = feature_removal_ids[feature_category]
    for feature_type in feature_removal:
        subset_features_to_drop = (
            feature_info_split_df
            .loc[
                feature_info_split_df.loc[:, feature_category] == feature_type,
                "full_feature_name"
            ]
            .tolist()
        )
        
        num_features_dropped = len(subset_features_to_drop)
        x_train_subset_df = x_train_df.drop(subset_features_to_drop, axis="columns")
        x_test_subset_df = x_test_df.drop(subset_features_to_drop, axis="columns")
        
        for cell_health_target in cell_health_targets:
            print(f"Now Training Target: {cell_health_target}")
            print(f"[Class] Sample Group Removal: {feature_category}; Dropping: {feature_type}\n")
            
            # Initialize class
            chp = CellHealthPredict(
                x_df=x_train_subset_df,
                y_df=y_train_df,
                parameters=parameters,
                estimator=estimator,
                n_folds=n_folds,
                cv_scoring=scoring,
                shuffle=False
            )

            # Fit model
            is_fit = chp.fit_cell_health_target(
                cell_health_target,
                y_transform=y_transform
            )

            # Training performance metrics
            metric_mse, metric_rtwo, y_true, y_pred = chp.get_performance(
                decision_function=decision_function,
                return_y=True,
                cell_line="all"
            )

            # Testing performance metrics
            metric_mse_test, metric_rtwo_test, y_test_true, y_test_pred = chp.get_performance(
                x_test=x_test_subset_df,
                y_test=y_test_df,
                decision_function=decision_function,
                return_y=True,
                data_fit_type="test",
                cell_line="all"
            )

            # Store results
            mse_df = (
                pd.concat([metric_mse, metric_mse_test], axis='rows')
                .reset_index(drop=True)
                .assign(
                    feature_category=feature_category,
                    feature_type_dropped=feature_type,
                    num_dropped=num_features_dropped
                )
            )
            
            rtwo_df = (
                pd.concat([metric_rtwo, metric_rtwo_test], axis='rows')
                .reset_index(drop=True)
                .assign(
                    feature_category=feature_category,
                    feature_type_dropped=feature_type,
                    num_dropped=num_features_dropped
                )
            )
            
            regression_results_list.append(mse_df)
            regression_results_list.append(rtwo_df)

Now Training Target: cc_all_high_h2ax
[Class] Sample Group Removal: feature_group; Dropping: Texture



/Users/gway/miniconda3/envs/cell-health/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/gway/miniconda3/envs/cell-health/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Now Training Target: cc_all_large_notround_polynuclear_mean
[Class] Sample Group Removal: feature_group; Dropping: Texture



/Users/gway/miniconda3/envs/cell-health/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/gway/miniconda3/envs/cell-health/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Now Training Target: cc_all_large_round_polyploid_mean
[Class] Sample Group Removal: feature_group; Dropping: Texture



...

/Users/gway/miniconda3/envs/cell-health/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/gway/miniconda3/envs/cell-health/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Now Training Target: vb_ros_back_mean
[Class] Sample Group Removal: compartment; Dropping: Nuclei



/Users/gway/miniconda3/envs/cell-health/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/gway/miniconda3/envs/cell-health/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Now Training Target: vb_ros_mean
[Class] Sample Group Removal: compartment; Dropping: Nuclei



/Users/gway/miniconda3/envs/cell-health/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/gway/miniconda3/envs/cell-health/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [7]:
# Compile the full regression results
full_regression_results_df = pd.concat(regression_results_list).reset_index(drop=True)

print(full_regression_results_df.shape)
full_regression_results_df.head(3)

(4200, 10)


,value,metric,target,data_fit,shuffle,y_transform,cell_line,feature_category,feature_type_dropped,num_dropped
0,0.418096,mse,cc_all_high_h2ax,train,shuffle_false,raw,all,feature_group,Texture,397
1,0.445491,mse,cc_all_high_h2ax,test,shuffle_false,raw,all,feature_group,Texture,397
2,0.407253,r_two,cc_all_high_h2ax,train,shuffle_false,raw,all,feature_group,Texture,397


In [8]:
# Save all results
full_regression_results_df.to_csv(output_file, sep="\t", index=False, compression="gzip")